In [ ]:
!pip3 install tensorflow_text>=2.0.0rc0

In [ ]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import pandas as pd
import os

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
INPUT_DIR = "/content/drive/MyDrive/project_data/"
df_test = pd.read_csv(os.path.join(INPUT_DIR, "test.csv"), header=0)
df_test = df_test.dropna(how="any")

Mounted at /content/drive


In [ ]:
categories = df_test.targetcat.unique()

In [ ]:
df = pd.DataFrame({'cat_name':categories})
df['cat_embeding'] = embed(df['cat_name'])

In [ ]:
df['cat_embeding'] = df['cat_embeding'].apply(lambda x: np.asarray(x))

In [ ]:
import joblib
joblib.dump(df, '/content/drive/MyDrive/project_data/df_cats.joblib')

['/content/drive/MyDrive/project_data/df_cats.joblib']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle
import numpy as np
sample = shuffle(df_test).head(10)
sample  

,description,title,targetcat
18554,EPDM Firestone GeoSmart (ГеоСмарт) – гидроизол...,"Мембрана EPDM Firestone GEOSMART для прудов 0,...",Аксессуары для прудов
12761,Цифровой угломер Shahe 5340-90B (4*90) с магни...,Цифровой угломер Shahe 5340-90B (4*90) с магни...,Угломеры
11224,Рукавички медичні нітрилові 200 шт в уп оглядо...,Перчатки медицинские смотровые НИТРИЛОВЫЕ 200 ...,Медицинские перчатки
12931,Тестер кабеля UNI-T UT681HDMI - профессиональн...,Тестер кабеля Uni-T UT681HDMI для интерфейса H...,Тестеры кабельные
4019,"Легкие, освежающие соки, нектары для детей и в...","Сок Морковь, Персик, Яблоко Bunio Marchew, Brz...",Детские соки и напитки
10548,В зимнее время нормальной работе автомобиля мо...,Зимняя накладка Fiat Doblo 2006-2011 Средняя м...,Накладки на бампер
14265,Если вам нужно управлять большими нагрузками у...,Умная GSM розетка Elgato 8 каналов IRON (DIN) ...,Умные розетки
7129,Описание ППКОП Лунь-25Е ППКОП поддерживает до ...,ППКОП Лунь-25Е,Централи
8012,"Бусины из Смолы, Цветок, Размер: 9x7мм, Отверс...",Бусины Monisto Смола 9x7мм Цвет: Микс Цветов 1...,Бусины
5431,"Кукурудза ТМ «ASP». Маса нетто 340 г, сорт вищ...",Кукурудза цукрова АСП ж/б 340 гр (4820049140415),Консервация


In [ ]:
sample['embed']  = embed(sample['description'])
sample['embed'] = sample['embed'].apply(lambda x: np.asarray(x))

In [ ]:
cos_sim = cosine_similarity(sample['embed'].tolist(), df['cat_embeding'].tolist())


In [ ]:
indexes = np.argmax(cos_sim, axis=1)
cats = df.loc[indexes]['cat_name']

In [ ]:
sample['cats'] = cats.tolist()

In [ ]:
sample

,description,title,targetcat,embed,cats
18554,EPDM Firestone GeoSmart (ГеоСмарт) – гидроизол...,"Мембрана EPDM Firestone GEOSMART для прудов 0,...",Аксессуары для прудов,"[-0.05214519, 0.02243376, -0.045297965, -0.048...",Гидроизоляция
12761,Цифровой угломер Shahe 5340-90B (4*90) с магни...,Цифровой угломер Shahe 5340-90B (4*90) с магни...,Угломеры,"[-0.013583375, 0.05920265, 0.017745733, -0.037...",Термометры и гигрометры
11224,Рукавички медичні нітрилові 200 шт в уп оглядо...,Перчатки медицинские смотровые НИТРИЛОВЫЕ 200 ...,Медицинские перчатки,"[-0.037558127, -0.059240017, 0.044681158, 0.06...",Ленты для рукоделия
12931,Тестер кабеля UNI-T UT681HDMI - профессиональн...,Тестер кабеля Uni-T UT681HDMI для интерфейса H...,Тестеры кабельные,"[-0.047342066, -0.04509521, -0.025346257, 0.05...",Тестеры кабельные
4019,"Легкие, освежающие соки, нектары для детей и в...","Сок Морковь, Персик, Яблоко Bunio Marchew, Brz...",Детские соки и напитки,"[-0.062259197, 0.011558943, -0.006491645, 0.00...",Наборы детского питания
10548,В зимнее время нормальной работе автомобиля мо...,Зимняя накладка Fiat Doblo 2006-2011 Средняя м...,Накладки на бампер,"[-0.051539455, -0.023917858, -0.031431932, -0....",Система отопления автомобиля
14265,Если вам нужно управлять большими нагрузками у...,Умная GSM розетка Elgato 8 каналов IRON (DIN) ...,Умные розетки,"[-0.053059768, 0.02032145, -0.051088665, 0.031...",Сигнализация
7129,Описание ППКОП Лунь-25Е ППКОП поддерживает до ...,ППКОП Лунь-25Е,Централи,"[-0.06080512, -0.010320031, -0.040508658, 0.01...",Разводные ключи
8012,"Бусины из Смолы, Цветок, Размер: 9x7мм, Отверс...",Бусины Monisto Смола 9x7мм Цвет: Микс Цветов 1...,Бусины,"[-0.041209675, 0.009330992, 0.0028819342, 0.03...","Мольберты, этюдники, палитры"
5431,"Кукурудза ТМ «ASP». Маса нетто 340 г, сорт вищ...",Кукурудза цукрова АСП ж/б 340 гр (4820049140415),Консервация,"[-0.041160572, 0.06776918, -0.027562425, -0.00...",Семена зерновых культур


### Lets do on the bigger test sample and check classification metrics

In [ ]:
sample = shuffle(df_test).head(4000) 
sample['embed']  = embed(sample['description'])
sample['embed'] = sample['embed'].apply(lambda x: np.asarray(x))

In [ ]:
cos_sim = cosine_similarity(sample['embed'].tolist(), df['cat_embeding'].tolist())

In [ ]:
indexes = np.argmax(cos_sim, axis=1)
cats = df.loc[indexes]['cat_name']
sample['prediction'] = cats.tolist()

In [ ]:
from sklearn.metrics import classification_report, f1_score
print(classification_report(sample['targetcat'], sample['prediction']))

                                                                      precision    recall  f1-score   support

                                                        3D и VR очки       0.00      0.00      0.00         1
                                                           3D панели       0.00      0.00      0.00         1
                                                        3D-раскраски       0.00      0.00      0.00         2
                                                            3D-ручки       0.00      0.00      0.00         1
                                                       3D-устройства       0.00      0.00      0.00         1
                                                     AHD видеокамеры       0.00      0.00      0.00         0
                                                         AV-ресиверы       0.00      0.00      0.00         3
                                                 Bluetooth-гарнитуры       0.00      0.00      0.00         0
         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
n = 3000
f1_scores = []
for g, sample in df_test.groupby(np.arange(len(df_test)) // n):
    sample['embed']  = embed(sample['description'])
    sample['embed'] = sample['embed'].apply(lambda x: np.asarray(x))
    cos_sim = cosine_similarity(sample['embed'].tolist(), df['cat_embeding'].tolist())
    indexes = np.argmax(cos_sim, axis=1)
    cats = df.loc[indexes]['cat_name']
    sample['prediction'] = cats.tolist()
    f1_scores.append((f1_score(sample['targetcat'], sample['prediction'],average='macro')))

In [ ]:
f1_scores

[0.10482862446510405,
 0.10275118283382637,
 0.11139602558314216,
 0.10370760090121574,
 0.10325976463771104,
 0.11058380425752994,
 0.1027166319230745]